In [1]:
import tweepy

In [147]:
import urllib3
import urllib3.request

import csv
import json
from pandas.io.json import json_normalize

import pandas as pd
pd.options.display.max_colwidth = 1000
import numpy as np

import nbimporter
import utility_functions as uf

In [148]:
from importlib import reload
reload(uf)

Importing Jupyter notebook from utility_functions.ipynb


<module 'utility_functions' from 'utility_functions.ipynb'>

In [124]:
####input your credentials here
consumer_key = '4ha4rLgP6Ci6fEZtaqttGTKoA'
consumer_secret = '5ckLaCgfTdfmWM7qS9f2w05pDCSIWRCTHlm7RLnKwK9tCWIz9P'
access_token = '602145669-jHmxtsl0wSZDFeZxi81GcTzYrD87dRBhF78ip0qo'
access_token_secret = 'YFLMmVVdcN4gb4KDX3MeOjbjxoKnnsFvjKxjRGMkkEZ5D'

In [185]:
#define inputs here
input_hashtag = "#giletjaune"
language = "fr"
since_date = "2018-03-07"
#n_tweets = 1

In [151]:
uf.db_init()
rt = 0

In [186]:
from twarc import Twarc
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret,tweet_mode= 'extended')
for tweet in t.search(input_hashtag, lang=language):
    with open('tweets_twarc/tweet'+str(tweet['id'])+'.json', 'w', encoding='utf8') as file:
        ####you need to create a 'tweets_twarc' folder
       
        json.dump(tweet, file)
    

In [187]:
## this cell will be deprecated when switching to utility function
rt = 0
#databases initialisations
def db_init():
    global df_users
    global df_tweets
    global df_hashtags
    global df_users_mentions
    global df_retweet_users
    
    #initialisation of databases, df files

    d_users = {'user_id': [],
               'name':[],
               'screen_name':[],
               'location':[],
               'description':[],
               'url':[],
               'followers_count':[],
               'friends_count':[],
               'listed_count':[],
               'created_at':[],
               'favourites_count':[],
               'geo_enabled':[],
               'verified':[],
               'statuses_count':[],
               'lang':[],
               'contributors_enabled':[]
              }
    df_users = pd.DataFrame(data=d_users)

    d_tweets = {'tweet_id': [],
                'created_at': [],
                'text': [],
                'truncated': [],
                'source': [],
                'in_reply_to_status_id': [],
                'in_reply_to_user_id': [],
                'in_reply_to_screen_name': [],
                'user_id': [],
                'geo': [],
                'coordinates': [],
                'place': [],
                'contributors': [],
                'is_quote_status': [],
                'retweet_count': [],
                'favorite_count': [],
                'favorited': [],
                'retweeted': [],
                'lang': []         
               }
    df_tweets = pd.DataFrame(data=d_tweets)


    d_hashtags={'hashtag': [], 'tweet_id': []}
    df_hashtags = pd.DataFrame(data=d_hashtags)

    d_users_mentions = {'tweet_id':[], 'user_id': [], 'screen_name':[], 'name':[]}
    df_users_mentions = pd.DataFrame(data=d_users_mentions)

    d_retweet_users = {'user_id':[], 'original_user_id':[], 'original_tweet_id':[]}
    df_retweet_users= pd.DataFrame(data=d_retweet_users)


# store all hashtags related to a tweet
def store_hashtag(tweet):
    global df_hashtags
    for raw_hash in tweet['entities']['hashtags']:
            df_hashtags = df_hashtags.append({'hashtag': raw_hash['text'] ,'tweet_id':tweet['id_str']}, ignore_index=True)

#store all user mentions contained in the tweet
def store_user_mentions(tweet):
    global df_users_mentions
    tweet_id = tweet['id_str']
    for raw_mention in tweet['entities']['user_mentions']:
        screen_name = raw_mention['screen_name']
        name = raw_mention['name']
        user_id = raw_mention['id_str']
        df_users_mentions = df_users_mentions.append({'tweet_id':tweet_id, 'user_id': user_id, 'screen_name':screen_name, 'name':name},ignore_index=True)

#store the author of the tweet        
def store_user(tweet):
    global df_users
    user = tweet['user']
    if user['id_str'] not in df_users['user_id'].values:
        
        d_user = {'user_id': user['id_str'],
                  'name':user['name'],
                  'screen_name':user['screen_name'],
                  'location':user['location'],
                  'description':user['description'],
                  'url':user['url'],
                  'followers_count':user['followers_count'],
                  'friends_count':user['friends_count'],
                  'listed_count':user['listed_count'],
                  'created_at':user['created_at'],
                  'favourites_count':user['favourites_count'],
                  'geo_enabled':user['geo_enabled'],
                  'verified':user['verified'],
                  'statuses_count':user['statuses_count'],
                  'lang':user['lang'],
                  'contributors_enabled':user['contributors_enabled']
                 }
        df_users = df_users.append(d_user,ignore_index=True)

#store the content of the tweet
def store_tweet(tweet):
    global df_tweets
    text = re.sub(r"http\S+", "", tweet['full_text']) ## Removes URLs from full text
    df_tweets = df_tweets.append({'tweet_id':  tweet['id_str'],
            'created_at': tweet['created_at'],
            'text': text,
            'truncated': tweet['truncated'],
            'source': tweet['source'],
            'in_reply_to_status_id': tweet['in_reply_to_status_id_str'],
            'in_reply_to_user_id': tweet['in_reply_to_user_id_str'],
            'in_reply_to_screen_name': tweet['in_reply_to_screen_name'],
            'user_id': tweet['user']['id_str'],
            'geo': tweet['geo'],
            'coordinates': tweet['coordinates'],
            'place': tweet['place'],
            'contributors': tweet['contributors'],
            'is_quote_status': tweet['is_quote_status'],
            'retweet_count': tweet['retweet_count'],
            'favorite_count': tweet['favorite_count'],
            'favorited': tweet['favorited'],
            'retweeted': tweet['retweeted'],
            'lang': tweet['lang'] },ignore_index=True)

#called when the tweet is a retweet: store the original author of the tweet as well as the user who retweeted it,
# and the original id of the tweet
def store_retweet_user(tweet):
    global df_retweet_users
    if is_a_retweet(tweet):
        df_retweet_users = df_retweet_users.append({'user_id':tweet['user']['id_str'],
                                                    'original_user_id':tweet['retweeted_status']['user']['id_str'],
                                                    'original_tweet_id':tweet['retweeted_status']['id_str']}, ignore_index=True)
##Never used
##T0D0: proper encapsulation
def store_influent_users(n):
    #n is the number of influent users return
    df_favorite_count = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).head()
    for i in range(n):
        d_influent_users = {'user_id':df_favorite_count[0], 
                            'name':[],
                            'retweet_count':[],
                            'favorite_count':[], 
                            'description':[] 
                           }
        
        
def is_a_retweet(tweet):
    #### returns a true if the tweet is a retweet, false if it's an original tweets 
    global rt
    rt += 1
    return tweet.get('retweeted_status',None) != None

def process_json_tweet(tweet):
    tweet = json.load(tweet)
    
    if is_a_retweet(tweet):
        store_retweet_user(tweet)
        
    else:  #store tweets only when they are original tweets
        store_tweet(tweet)
        store_user(tweet)
        store_hashtag(tweet)
        store_user_mentions(tweet)
    

In [189]:
db_init()

In [192]:
#### loading the databases
t= 0
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
import os

path_to_json = 'tweets_twarc/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for json_file in json_files:
    
    with open(path_to_json+json_file) as tweet:
        t+=1
        process_json_tweet(tweet)
        
        
        

In [193]:
df_tweets.head(10);
print(rt)
print(t)

22158
12080


In [194]:
#tweet example
df_tweets['text'].iloc[0]

'[ #GiletsJaunes #ActeXVIII ] \n\n⚡️Plusieurs centaines de #GiletsJaunes défilent dans les rues de #Paris ❗️\n\n📢🗣«\xa0#Macron démission / #Castaner en prison !\xa0»\n\n#giletjaune #Acte18bis #EmmanuelMacron #acteXVIIIbis #manifestations #manifestation #Greve19mars #France #politique \n\n⬇️📹 '

In [195]:
d = pd.to_datetime(df_tweets['created_at'])
print(d.min())
print(d.max())
print(len(df_tweets.index)) 

2019-03-19 18:25:03+00:00
2019-03-29 14:29:31+00:00
2002


In [196]:
#primary keys are ok if both return false
print(df_tweets.duplicated('tweet_id').any())
print(df_users.duplicated('user_id').any())

False
False


# Aggregations


## Top users

In [197]:
df_most_favorited_users = df_tweets.groupby(['user_id'])['favorite_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_retweeted_users = df_tweets.groupby(['user_id'])['retweet_count'].agg('sum').sort_values(ascending=False).reset_index().head(15)
df_most_favorited_users['favorite_count'] = df_most_favorited_users['favorite_count'].astype('int32')
df_most_retweeted_users['retweet_count'] = df_most_retweeted_users['retweet_count'].astype('int32')

df_most_favorited_users['user_id'] = df_most_favorited_users['user_id'].astype(object)
df_most_retweeted_users['user_id'] = df_most_retweeted_users['user_id'].astype(object)

In [198]:
df_most_favorited_users_with_info = df_most_favorited_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users.set_index('user_id').join(df_users.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [199]:
df_most_retweeted_users_with_info.head(15);

In [200]:
df_most_favorited_users_with_info.head(15);

In [201]:
df_most_retweeted_users_with_info.reset_index()
df_most_favorited_users_with_info.reset_index();

In [202]:
print(df_most_retweeted_users_with_info['screen_name'])
print(df_most_favorited_users_with_info['screen_name'])

user_id
1028668294751039488     GiletsJaunesFr
784636611183214592        FranceNews24
1083279881209896966     LorentzMathias
1084126493788332033         AA36514431
435224364                    Callystor
155606403                  LEDOUAISIEN
338955445                    tcabarrus
29707536                     quatremer
4883112166                mojitobatbat
1062458079470600194    collCartonJaune
861352966653173760             HRyssen
829280595230486528      Maraine_Lapine
859534493111156736         ion_positon
859407061230252032     Circonscripti18
808440877865717766     GaetanEscorbiac
Name: screen_name, dtype: object
user_id
1028668294751039488     GiletsJaunesFr
29707536                     quatremer
784636611183214592        FranceNews24
1083279881209896966     LorentzMathias
155606403                  LEDOUAISIEN
338955445                    tcabarrus
861352966653173760             HRyssen
435224364                    Callystor
1084126493788332033         AA36514431
107180560099513

In [203]:
df_tweets_by_user = df_tweets.groupby(['user_id']).size().to_frame().reset_index()
df_most_favorited_users_with_info = df_most_favorited_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('favorite_count',ascending=False)
df_most_retweeted_users_with_info = df_most_retweeted_users_with_info.join(df_tweets_by_user.set_index('user_id')).drop_duplicates().sort_values('retweet_count',ascending=False)

In [204]:
df_most_favorited_users_with_info;

In [205]:
top_influencers_fav = df_most_favorited_users_with_info[['screen_name','description', 'followers_count', 'friends_count','favorite_count',0]]
top_influencers_rt = df_most_retweeted_users_with_info[['screen_name','description', 'followers_count', 'friends_count','retweet_count',0]]
top_influencers_fav.columns = ["Name","Description","Number of followers","Number of friends","Number of favorites","Number of tweets"]
top_influencers_rt.columns = ["Name","Description","Number of followers","Number of friends","Number of retweets","Number of tweets"]
top_influencers_fav = top_influencers_fav.reset_index().drop(columns = ["user_id"])
top_influencers_rt = top_influencers_rt.reset_index().drop(columns = ["user_id"]);
top_influencers_fav['Number of followers'] = top_influencers_fav['Number of followers'].astype(int)
top_influencers_fav['Number of friends'] = top_influencers_fav['Number of friends'].astype(int)
top_influencers_rt['Number of followers'] = top_influencers_rt['Number of followers'].astype(int)
top_influencers_rt['Number of friends'] = top_influencers_rt['Number of friends'].astype(int)

In [206]:
#add ratios
top_influencers_fav.loc[:,'Like Tweet Ratio'] = top_influencers_fav.loc[:,"Number of favorites"]/top_influencers_fav.loc[:,"Number of tweets"]
top_influencers_rt.loc[:,'Retweet Tweet Ratio'] = top_influencers_rt.loc[:,"Number of retweets"]/top_influencers_rt.loc[:,"Number of tweets"]

In [207]:
top_influencers_fav_names = top_influencers_fav['Name'].values
top_influencers_rt_names = top_influencers_rt['Name'].values
print(top_influencers_rt_names)

['GiletsJaunesFr' 'FranceNews24' 'LorentzMathias' 'AA36514431' 'Callystor'
 'LEDOUAISIEN' 'tcabarrus' 'quatremer' 'mojitobatbat' 'collCartonJaune'
 'HRyssen' 'Maraine_Lapine' 'ion_positon' 'Circonscripti18'
 'GaetanEscorbiac']


## Top hashtags for this input hashtags and by users

In [208]:
top_influencers_fav.style.set_properties(**{'text-align': 'left'});

In [209]:
df_tweets_hashtags = df_hashtags.set_index('tweet_id').join(df_tweets.set_index('tweet_id'))

In [210]:
df_tw_hash_user = df_tweets_hashtags.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))

In [211]:
#df_aggretated = df_tw_hash_user.groupby(['user_id','hashtag']).size().reset_index()
df_aggretated = df_tw_hash_user.groupby(['screen_name','hashtag']).size().reset_index()

In [212]:
df_aggretated.columns = ["Name", "#", "Used"]
df_aggretated;

In [213]:
#useful encapsulation, aint it?
def get_tweet_by_userid(user_id):
    df_tweets[df_tweets['user_id'] == 'user_id']

def get_tweet_by_username(name):
    df_tweets_user_info = df_tweets.set_index('user_id').join(df_users.drop(columns=['created_at', 'lang']).set_index('user_id'))
    df_tweets_user_info = df_tweets_user_info[df_tweets_user_info['screen_name'] == name]
    df_tweets_user_info = df_tweets_user_info[['tweet_id','text','created_at', 'favorite_count', 'retweet_count']]
    return df_tweets_user_info

def get_related_hashtags_by_username(name):
    d_name = df_aggretated[df_aggretated['Name'] == name][['#',"Used"]].sort_values("Used", ascending = False)
    return d_name

In [216]:
get_tweet_by_username('GiletsJaunesFr')

,tweet_id,text,created_at,favorite_count,retweet_count
user_id,,,,,
1028668294751039488,1108163259931205637,"Répartition des fruits du travail ! ""Rothschild and Co"" présente ici son ""Rapport annuel 2017"". Énormes bénéfices en pourcentage par rapport aux bénéfices déjà conséquent de 2016 ... L'hémorragie est permanente ! \n\n#GiletJaune #GiletsJaunes #Finance #banque",Wed Mar 20 00:28:04 +0000 2019,11.0,18.0
1028668294751039488,1108261628309573632,"Parler d'une manifestation à venir vous en rend responsable ! Y marcher vous en rend complice ! etc ... Choisir des têtes, les criminaliser pour l'exemple ... et ensuite ?\n#GiletJaune #GiletsJaunes #YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019",Wed Mar 20 06:58:57 +0000 2019,82.0,53.0
1028668294751039488,1108296971641597952,Acte 18 : Micro trottoir auprès des Gilets Jaunes.\n\n#GiletJaune #GiletsJaunes #YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #assembleenationale #senat #Europeennes2019 #France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews,Wed Mar 20 09:19:23 +0000 2019,21.0,12.0
1028668294751039488,1108313029769347072,"C'est un tel privilège d'être ministre, un tel honneur de servir ce pays, que l'on peut faire un effort d'exemplarité !\n#GiletJaune #GiletsJaunes #YellowVests #Gelbwesten #Europeennes2019 #assembleenationale #senat #Europeennes2019 #France2 #BFM #RTFrance #LCI #Cnews",Wed Mar 20 10:23:12 +0000 2019,34.0,35.0
1028668294751039488,1108432646458232832,Quand la castagne devient l'unique projet politique ...\n#GiletJaune #GiletsJaunes #RIC #assembleenationale #senat\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews\n#YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #ecologie #Climat,Wed Mar 20 18:18:31 +0000 2019,58.0,38.0
1028668294751039488,1108652526038147072,"Mutilés après mutilés, les médias sont au diapason, l'omerta sur les violences d'état est totale, la non-réponse politique aux frigos vides est criante, le peuple est alors criminalisé, la répression est totale.\n#GiletJaune #GiletsJaunes #YellowVests #Gelbwesten #Europeennes2019",Thu Mar 21 08:52:14 +0000 2019,743.0,606.0
1028668294751039488,1108825050071945216,"Pris par le temps, mais également par l'argent, les premiers tests de l' ""Exterminator GJ"" seront menés in situ, samedi prochain, boutique du souvenir, Arc de Triomphe.\n#GiletJaune #GiletsJaunes #Acte18 #Acte19\n#YellowVests #chalecosamarillos #Gelbwesten #armee #Europeennes2019",Thu Mar 21 20:17:47 +0000 2019,88.0,56.0
1028668294751039488,1109030876665602050,"Toute revendication sociale est étouffée dans l’œuf, grâce à l'action toujours incontrôlable des Black Blocs , un mouvement né en Allemagne dans les années 80, il y a plus de 35 ans ! Le 17 nov 2018 , il était simplement question de cela :\n#GiletJaune #GiletsJaunes #RIC",Fri Mar 22 09:55:40 +0000 2019,69.0,39.0
1028668294751039488,1109073160639598595,"Privatisons, privatisons ...\n#GiletJaune #GiletsJaunes #RIC #assembleenationale #senat\n#France2 #BFM #RTFrance #LCI #Cnews #CNN #CBSNews\n#YellowVests #chalecosamarillos #coletesamarelos #Gelbwesten #Europeennes2019 #ecologie #Climat #Privatisation",Fri Mar 22 12:43:41 +0000 2019,27.0,17.0


In [217]:
#output 
for name in top_influencers_rt_names:
    df_rel = get_related_hashtags_by_username(name)
    print(name)
    df_rel.to_csv('top_hashtags_by_'+name+'.csv',index=False)
    df_rel=[]

GiletsJaunesFr
FranceNews24
LorentzMathias
AA36514431
Callystor
LEDOUAISIEN
tcabarrus
quatremer
mojitobatbat
collCartonJaune
HRyssen
Maraine_Lapine
ion_positon
Circonscripti18
GaetanEscorbiac


In [219]:
get_related_hashtags_by_username('GiletsJaunesFr')

,#,Used
1506,GiletJaune,28
1507,GiletsJaunes,27
1515,YellowVests,22
1523,chalecosamarillos,20
1499,Europeennes2019,19
1502,Gelbewesten,15
1525,droitsdelhomme,10
1488,Amnestyinternational,10
1514,ViolencesPolicieres,10
1489,BFM,9


# NLP


In [220]:
import spacy

In [221]:
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import fr_core_news_sm
nlp = fr_core_news_sm.load()
#!python -m spacy download fr_core_web_lg

In [222]:
doc = nlp(df_tweets["text"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [223]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [224]:
review = str(" ".join([i.lemma_ for i in doc]))

In [225]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [226]:
for i in nlp(review):
    print(i,"=>",i.pos_)

" => PUNCT
ce => DET
être => AUX
en => PRON
faire => VERB
ne => ADV
importer => VERB
quoi => PRON
, => PUNCT
que => SCONJ
le => DET
on => PRON
devenir => VERB
ne => ADV
importer => VERB
qui => PRON
" => PUNCT
on => PRON
accréditer => VERB
ce => DET
phrase => NOUN
à => ADP
@remigaillard => ADV
_ => ADP
   => SPACE
mais => CCONJ
désormais => ADV
ce => DET
être => AUX
un => DET
petit => ADJ
peu => ADV
le => DET
histoire => NOUN
de => ADP
@ccastaner => VERB
# => ADJ
manifestation => NOUN
# => ADJ
giletjaun => ADJ
# => ADJ
Castaner => PROPN


In [227]:
parser = French()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [228]:
tqdm.pandas()
df_tweets["processed_text"] = df_tweets["text"].progress_apply(spacy_tokenizer)

100%|██████████| 2002/2002 [00:04<00:00, 413.22it/s]


In [229]:
df_tweets.head(10)["processed_text"];

In [230]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words=stopwords, lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df_tweets["processed_text"])

In [231]:
##How many topics do you want to find??
NUM_TOPICS = 6

In [232]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [233]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [234]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [235]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [236]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('giletsjaunes', 135.04456038453407), ('acte', 123.44535435237268), ('france', 101.9035437176134), ('macron', 81.08229676679088), ('yellowvests', 68.92279448226014), ('jaunes', 61.67862311328099), ('gilets', 60.880551758807854), ('bfmtv', 52.64254404672329), ('toulouse', 52.01484036435735), ('actexix', 51.05639264548391)]
Topic 1:
[('france', 67.97352487950637), ('lrem', 50.40305913502078), ('ric', 46.82168658957686), ('faire', 45.7342997231904), ('aller', 42.38774125466867), ('strat', 33.83313014505813), ('incomp', 31.833394750798043), ('coup', 31.80235045123092), ('militaire', 30.993557661997677), ('franceinsoumise', 30.1984553717635)]
Topic 2:
[('dan', 104.61998113229721), ('plaire', 89.44857464706965), ('faire', 88.77530316845723), ('giletsjaunes', 80.30827661399812), ('france', 66.16167027626331), ('mettre', 56.05891985152986), ('fran', 52.40244700752155), ('ais', 49.491059281630626), ('emmanuelmacron', 49.115621918653936), ('bon', 47.18571495686509)]
Topic 3:

In [237]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('mars', 3.791784848710898), ('samedi', 1.9561652453488696), ('stopgiletsjaunes', 1.7066457513057869), ('actexix', 1.6421733143403803), ('blackbloc', 1.4138477570118202), ('noborder', 1.3407061003471292), ('antifa', 1.2824997516480934), ('direct', 1.2706391266580561), ('giletsjaunes', 1.1988009887581557), ('acte', 1.189375388946127)]
Topic 1:
[('giletsjaunes', 5.619828005169278), ('yellowvests', 0.5007865899902789), ('europeennes', 0.46628414243489247), ('actexix', 0.4434437016316576), ('gilets', 0.41489695690597256), ('jaunes', 0.41320032303255905), ('giletsjaunesparis', 0.4013892655140205), ('paris', 0.26748002595159776), ('emmanuelmacron', 0.2665176898892801), ('manifestation', 0.25902503518308534)]
Topic 2:
[('faire', 4.0137710733634115), ('plaire', 1.6589902479126828), ('pouvoir', 1.0574214529693802), ('aller', 0.8837114989572766), ('vouloir', 0.5975673569347841), ('voir', 0.4383109316190568), ('falloir', 0.4232894002217077), ('ais', 0.40670563621776346), ('fr

In [238]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('giletsjaunes', 0.5505398344964729), ('acte', 0.46845544793115), ('mars', 0.3177454597845922), ('actexix', 0.2799903551839373), ('samedi', 0.18350774750462753), ('macron', 0.14202872980160358), ('stopgiletsjaunes', 0.13843684695456035), ('nice', 0.13276735267378173), ('france', 0.11767704439466756), ('blackbloc', 0.10860015727075555)]
Topic 1:
[('mars', 0.44035187196511083), ('samedi', 0.206540009979252), ('stopgiletsjaunes', 0.197172181387248), ('blackbloc', 0.1743340107589809), ('noborder', 0.16771675539288658), ('antifa', 0.1599843511417086), ('direct', 0.15143304755523174), ('yellowvest', 0.13719267031804996), ('actexix', 0.13145619738881084), ('yellowjackets', 0.113614039612253)]
Topic 2:
[('acte', 0.5156683076678232), ('giletsjaunes', 0.14331460738117782), ('gilets', 0.07322202500359949), ('actexx', 0.07062065985272749), ('jaunes', 0.06429265125396552), ('actexix', 0.06118340434486513), ('yellowvests', 0.04778978729880896), ('castaner', 0.04616914060257458),

In [239]:
# Transforming an individual sentence
text = spacy_tokenizer("Les gilets jaune tous unis contre Macron. Tous dans la rue jusqu'à la démission")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.02090125 0.02090845 0.33523636 0.43413618 0.02103398 0.16778377]


In [240]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      -7.502934  44.710052       1        1  22.332860
2     -25.632648   3.315551       2        1  20.354610
4      47.128963  -3.888149       3        1  17.748955
0      11.918715  11.536210       4        1  14.875730
3      37.468178  40.256969       5        1  12.590274
1       8.130635 -26.717463       6        1  12.097571, topic_info=     Category        Freq                  Term       Total  loglift  logprob
6     Default  445.000000                  acte  445.000000  30.0000  30.0000
494   Default  121.000000                 jaune  121.000000  29.0000  29.0000
587   Default  201.000000                  mars  201.000000  28.0000  28.0000
657   Default  127.000000                  nice  127.000000  27.0000  27.0000
423   Default   89.000000                 gilet   89.000000  26.0000  26.0000
8     Default  241.000000               actexix  241.000000  25.0000  25.0000
757   Default  155.000000               pouvoir  155.000000  24.0000  24.0000
751   Default   78.000000                porter   78.000000  23.0000  23.0000
392   Default  257.000000                france  257.000000  22.0000  22.0000
1022  Default   67.000000           yellowvests   67.000000  21.0000  21.0000
866   Default  149.000000                samedi  149.000000  20.0000  20.0000
908   Default  101.000000      stopgiletsjaunes  101.000000  19.0000  19.0000
742   Default   91.000000              policier   91.000000  18.0000  18.0000
729   Default  167.000000                plaire  167.000000  17.0000  17.0000
495   Default   69.000000                jaunes   69.000000  16.0000  16.0000
935   Default   68.000000                   ter   68.000000  15.0000  15.0000
551   Default   66.000000                  lrem   66.000000  14.0000  14.0000
425   Default   77.000000                gilets   77.000000  13.0000  13.0000
517   Default   48.000000                   lci   48.000000  12.0000  12.0000
949   Default   55.000000              toulouse   55.000000  11.0000  11.0000
836   Default   91.000000                   ric   91.000000  10.0000  10.0000
108   Default   61.000000             blackbloc   61.000000   9.0000   9.0000
699   Default  109.000000                 paris  109.000000   8.0000   8.0000
346   Default   61.000000           europeennes   61.000000   7.0000   7.0000
111   Default   48.000000               blesser   48.000000   6.0000   6.0000
726   Default   47.000000                 place   47.000000   5.0000   5.0000
102   Default   68.000000                 bfmtv   68.000000   4.0000   4.0000
390   Default   92.000000                  fran   92.000000   3.0000   3.0000
661   Default   54.000000              noborder   54.000000   2.0000   2.0000
427   Default  653.000000          giletsjaunes  653.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
126    Topic6   10.059299            bouteflika   10.865530   2.0351  -5.4514
746    Topic6    9.974733                pondre   10.775281   2.0350  -5.4598
557    Topic6    9.835442  macroncomplicedupire   10.638400   2.0337  -5.4739
569    Topic6    9.676860                   mal   10.475796   2.0328  -5.4901
561    Topic6   18.580218              macronie   22.563045   1.9179  -4.8378
551    Topic6   46.481362                  lrem   66.612621   1.7523  -3.9208
395    Topic6   27.848812       franceinsoumise   36.913403   1.8304  -4.4331
616    Topic6   28.582050             militaire   39.823598   1.7805  -4.4071
401    Topic6   24.085512                frexit   35.164550   1.7337  -4.5783
836    Topic6   43.178644                   ric   91.310811   1.3632  -3.9945
595    Topic6   14.448359                   mec   18.597392   1.8597  -5.0893
389    Topic6   15.572082                foutre   21.076427   1.8095  -5.0144
392    Topic6   62.684727                france  257.866448   0.6978  -3.6218

Importing Jupyter notebook from utility_functions.ipynb


In [74]:
db_init()